## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [Spanish Medical LLM 24B]()
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

- **Model**: `JSL-Spanish-Medical-LLM-24B`
- **Model Description**: Spanish Medical LLM 24B

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [ ]:
import os
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

In [3]:
model_name = "JSL-Spanish-Medical-LLM-24B"

real_time_inference_instance_type = "ml.g5.48xlarge"
batch_transform_inference_instance_type = "ml.g5.48xlarge"

## 2. Create a deployable model from the model package.

In [4]:
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn, 
    sagemaker_session=sagemaker_session, 
)

## Model Configuration Documentation  

### Default Configuration  
The container comes with the following default configurations:  

| Parameter                  | Default Value              | Description                                                                     |  
|----------------------------|----------------------------|---------------------------------------------------------------------------------|  
| **`host`**                 | `0.0.0.0`                  | Hostname for the server                                                         |  
| **`port`**                 | `8080`                     | Port number used by the server                                                  |  
| **`tensor_parallel_size`** | Auto                       | Automatically set to the number of available GPUs                               |   
| **`tokenizer_mode`**       | `auto`                     | Automatically infers tokenizer configuration                                    |    

### Hardcoded Settings  
The following settings are hardcoded in the container and cannot be changed:  

| Parameter       | Value           | Description                           |  
|-----------------|-----------------|---------------------------------------|  
| **`model`**     | `/opt/ml/model` | Model path where SageMaker mounts the model |  

### Configurable Environment Variables  
You can customize the vLLM server by setting environment variables when creating the model.  

**Any parameter from the [vLLM documentation](https://docs.vllm.ai/en/latest/serving/openai_compatible_server.html#vllm-serve) can be set using the corresponding environment variable with the `SM_VLLM_` prefix.**  

The container uses a script similar to the [SageMaker entrypoint example](https://docs.vllm.ai/en/latest/examples/sagemaker_entrypoint.html) from the vLLM documentation to convert environment variables to command-line arguments.  

---  

## Input Format  

### 1. Chat Completion  

#### Example Payload  
```json  
{  
    "model": "/opt/ml/model",  
    "messages": [  
        {"role": "system", "content": "Eres un asistente médico útil."},  
        {"role": "user", "content": "¿Qué debo hacer si tengo fiebre y dolores corporales?"}  
    ],  
    "max_tokens": 1024,  
    "temperature": 0.7  
}  
```  

For additional parameters:  
- [ChatCompletionRequest](https://github.com/vllm-project/vllm/blob/v0.8.5/vllm/entrypoints/openai/protocol.py#L232)  
- [OpenAI's Chat API](https://platform.openai.com/docs/api-reference/chat/create)  

---  

### 2. Text Completion  

#### Single Prompt Example  
```json  
{  
    "model": "/opt/ml/model",  
    "prompt": "¿Qué debo hacer si tengo fiebre y dolores corporales?",  
    "max_tokens": 512,  
    "temperature": 0.6  
}  
```  

#### Multiple Prompts Example  
```json  
{  
    "model": "/opt/ml/model",  
    "prompt": [  
        "¿Cómo puedo mantener una buena salud renal?",  
        "¿Cuáles son los síntomas de la hipertensión?"  
    ],  
    "max_tokens": 512,  
    "temperature": 0.6  
}  
```  

Reference:  
- [CompletionRequest](https://github.com/vllm-project/vllm/blob/v0.8.5/vllm/entrypoints/openai/protocol.py#L730)  
- [OpenAI's Completions API](https://platform.openai.com/docs/api-reference/completions/create)  

---  

### Important Notes:
- **Streaming Responses:** Add `"stream": true` to your request payload to enable streaming
- **Model Path Requirement:** Always set `"model": "/opt/ml/model"` (SageMaker's fixed model location)

## 3. Create an SageMaker Endpoint

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [ ]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
    model_data_download_timeout=3600
)

### 3.1 Real-time inference via Amazon SageMaker Endpoint

#### Initial setup

In [9]:
prompt1 = "¿Qué debo hacer si tengo fiebre y dolores corporales?"

prompt2 = """Una mujer embarazada de 23 años, con 22 semanas de gestación, se presenta con ardor al orinar. Ella menciona que comenzó hace un día y ha ido empeorando a pesar de beber más agua y tomar extracto de arándano. Por lo demás, se siente bien y está siendo seguida por un médico para su embarazo. Su temperatura es 97.7°F (36.5°C), la presión arterial es 122/77 mmHg, el pulso es de 80/min, las respiraciones son 19/min, y la saturación de oxígeno es 98% en aire ambiente. El examen físico es notable por la ausencia de dolor en el ángulo costovertebral y un útero grávido.

¿Cuál de los siguientes es el mejor tratamiento para esta paciente?
A: Ampicilina
B: Ceftriaxona
C: Ciprofloxacino
D: Doxiciclina
E: Nitrofurantoína
"""

prompts = [
    "¿Cómo puedo mantener una buena salud renal?",
    "¿Cuáles son los síntomas de la hipertensión?"
]

In [10]:
system_prompt = "Actúe como asistente médico y responda únicamente en español. Ofrezca respuestas con encabezados numéricos estructurados y jerárquicos. Cuando sea necesario, incluya una conclusión clara y concisa que resuma los puntos clave o las recomendaciones."

In [11]:
def invoke_realtime_endpoint(record):

    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType="application/json",
        Accept="application/json",
        Body=json.dumps(record),
    )

    return json.load(response["Body"])

#### Chat Completion

In [12]:
input_data = {
    "model": "/opt/ml/model",
    "messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt1},
    ],
    "max_tokens": 2048,
    "temperature": 0.15,
}

result = invoke_realtime_endpoint(input_data)
output_content = result['choices'][0]['message']['content']
print(output_content)

### 1. Evaluación Inicial
- **Medición de la Temperatura:** Utilice un termómetro para medir su temperatura corporal. La fiebre se define generalmente como una temperatura superior a 38°C (100.4°F).
- **Observación de Síntomas:** Preste atención a otros síntomas que pueda estar experimentando, como dolor de cabeza, dolor de garganta, tos, o dificultad para respirar.

### 2. Medidas Inmediatas
- **Hidratación:** Beba abundantes líquidos para mantenerse hidratado. El agua, caldos, y bebidas electrolíticas son buenas opciones.
- **Descanso:** Descansar es crucial para que su cuerpo pueda combatir la infección.
- **Medicamentos:** Puede tomar medicamentos antipiréticos como el paracetamol o el ibuprofeno para reducir la fiebre y aliviar los dolores corporales. Siga las indicaciones del envase y no exceda la dosis recomendada.

### 3. Monitoreo y Observación
- **Control de la Temperatura:** Continúe midiendo su temperatura cada pocas horas para asegurarse de que no esté aumentando.
- **Obse

#### Text Completion

In [14]:
input_data ={
        "model": "/opt/ml/model",
        "prompt": prompt2,
        "max_tokens": 2048,
        "temperature": 0.15,
    }

result = invoke_realtime_endpoint(input_data)
output_text = result['choices'][0]['text']
print(output_text)

F: Trimethoprim-sulfametoxazol

La respuesta correcta es E: Nitrofurantoína.

La paciente presenta síntomas de infección del tracto urinario (ITU), que es común en mujeres embarazadas debido a cambios hormonales y anatómicos que facilitan la colonización bacteriana. La nitrofurantoína es el tratamiento de primera línea para las ITU no complicadas en mujeres embarazadas debido a su seguridad y eficacia. Es importante evitar medicamentos como el ciprofloxacino y la doxiciclina debido a sus efectos teratogénicos. Aunque la ampicilina y la ceftriaxona son opciones seguras durante el embarazo, no son el tratamiento de primera línea para ITU no complicadas. El trimethoprim-sulfametoxazol debe evitarse en el primer trimestre debido al riesgo de defectos congénitos.


### 3.2 Real-time inference response as a stream via Amazon SageMaker Endpoint

In [16]:
def invoke_streaming_endpoint(record):
    try:
        response = sm_runtime.invoke_endpoint_with_response_stream(
            EndpointName=model_name,
            Body=json.dumps(record),
            ContentType="application/json",
            Accept="text/event-stream"
        )

        for event in response["Body"]:
            if "PayloadPart" in event:
                chunk = event["PayloadPart"]["Bytes"].decode("utf-8")
                if chunk.startswith("data:"):
                    try:
                        data = json.loads(chunk[5:].strip())
                        if "choices" in data and len(data["choices"]) > 0:
                            choice = data["choices"][0]
                            if "text" in choice:
                                yield choice["text"]
                            elif "delta" in choice and "content" in choice["delta"]:
                                yield choice["delta"]["content"]

                    except json.JSONDecodeError:
                        continue 
            elif "ModelStreamError" in event:
                error = event["ModelStreamError"]
                yield f"\nStream error: {error['Message']} (Error code: {error['ErrorCode']})"
                break
            elif "InternalStreamFailure" in event:
                failure = event["InternalStreamFailure"]
                yield f"\nInternal stream failure: {failure['Message']}"
                break
    except Exception as e:
        yield f"\nAn error occurred during streaming: {str(e)}"

#### Chat Completion

In [17]:
payload = {
    "model": "/opt/ml/model",
    "messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt1}
    ],
    "max_tokens": 2048,
    "temperature": 0.15,
    "stream": True
}

for chunk in invoke_streaming_endpoint(payload):
    print(chunk, end="", flush=True)

1 **Evaluación Inicial**
   - **Medición de la Temperatura**: Utilice un termómetro para medir su temperatura corporal. La fiebre se define generalmente como una temperatura superior a 38°C (100.4°F).
   - **Observación de Síntomas**: Preste atención a otros síntomas que pueda estar experimentando, como dolor de cabeza, dolor de garganta, tos, o dificultad para respirar.

2. **Medidas Inmediatas**
   - **Hidratación**: Beba abundantes líquidos para mantenerse hidratado. El agua, caldos y bebidas electrolíticas son buenas opciones.
   - **Descanso**: Descansar es crucial para que su cuerpo pueda combatir la infección.
   - **Medicamentos**: Puede tomar medicamentos antipiréticos como el paracetamol o el ibuprofeno para reducir la fiebre y aliviar los dolores corporales. Siga las indicaciones del envase y no exceda la dosis recomendada.

3. **Monitoreo de Síntomas**
   - **Observación Continua**: Monitoree su temperatura y otros síntomas regularmente.
   - **Signos de Alarma**: Busque at

#### Text Completion

In [18]:
payload = {
    "model": "/opt/ml/model",
    "prompt": prompt2,
    "max_tokens": 2048,
    "temperature": 0.15,
    "stream": True
}

for chunk in invoke_streaming_endpoint(payload):
    print(chunk, end="", flush=True)

F: Trimetoprim/sulfametoxazol

La correcta es E: Nitrofurantoína.

La paciente presenta síntomas de infección del tracto urinario (ITU). La nitrofurantoína es el de primera línea para las ITU no complicadas durante el embarazo. Es segura para el feto y efectiva contra la mayoría de las bacterias que causan ITU. Las opciones A, B, C, D y F no son recomendadas durante el embarazo debido a sus efectos potenciales sobre el feto.

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [20]:
validation_json_file_name1 = "input1.json"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/"


def write_and_upload_to_s3(input_data, file_name):
    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/{file_name}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [21]:
input_json_data1 = json.dumps(
    {
        "model": "/opt/ml/model",
        "prompt": prompts,
        "max_tokens": 2048,
        "temperature": 0.15,
    }, 
    ensure_ascii=False
)

write_and_upload_to_s3(input_json_data1, f"{validation_json_file_name1}")

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path,
)
transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [24]:
from urllib.parse import urlparse

def retrieve_json_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)
    result = json.loads(response["Body"].read().decode("utf-8"))

    for idx, choice in enumerate(result.get("choices", [])):
        print(f"Response {idx + 1}:\n{choice.get('text', '')}\n{'=' * 75}")

In [25]:
retrieve_json_output_from_s3(validation_json_file_name1)

Response 1:
 La salud renal es fundamental para el bienestar general del cuerpo. Los riñones son responsables de filtrar los desechos y el exceso de líquidos de la sangre, regular los niveles de electrolitos y producir hormonas esenciales. Aquí hay algunas recomendaciones para mantener una buena salud renal:

1. **Hidratación adecuada**: Beber suficiente agua ayuda a los riñones a eliminar los desechos y las toxinas del cuerpo. La cantidad recomendada puede variar, pero generalmente se sugiere entre 1.5 y 2 litros de agua al día, dependiendo de la actividad física y el clima.

2. **Dieta equilibrada**: Consumir una dieta rica en frutas, verduras, granos enteros y proteínas magras puede ayudar a mantener la salud renal. Limitar el consumo de sal, azúcar y grasas saturadas también es beneficioso.

3. **Control del peso**: Mantener un peso saludable reduce la presión sobre los riñones y disminuye el riesgo de enfermedades renales.

4. **Evitar el tabaco y el alcohol**: Fumar y el consumo 

Congratulations! You just verified that the batch transform job is working as expected. Since the model is not required, you can delete it. Note that you are deleting the deployable model. Not the model package.

In [ ]:
model.delete_model()

### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

